In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

from numpy import pi 
from sympy import Function, simplify

# from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

from sympy.utilities.lambdify import implemented_function

from sympy import lambdify

from sympy.utilities.lambdify import lambdastr

# 1 Viscous Swimmer

## 1. Variable Length Jacobian

### Symbolic variables for our robot configuration

In [2]:
(theta_1,
 theta_2,
 theta_3,
 L_1,
 L_2,
 L_3,
 B,
 t) = sp.symbols("""
                    theta_1,
                    theta_2,
                    theta_3,
                    l_1,
                    l_2,
                    l_3,
                    B,
                    t
                    """,real=True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

### Generic 2D Transformation Function

In [3]:
def T(theta,L):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), L*sp.cos(theta)],
                      [sp.sin(theta),  sp.cos(theta), L*sp.sin(theta)],
                      [            0,              0,               1]])

### Zero Position

In [4]:
M = sp.Matrix([[0],[0],[1]])

### Formward Kinematics 

In [5]:
# Kinematics of the First Joint
def T1(theta_1,L_1,s):
    return T(theta_1,s*L_1) 

# Kinematics of the Second Joint
def T2(theta_1,theta_2,L_1,L_2,s):
    return T(theta_1,L_1) @ T(theta_2,s*L_2) 

# Kinematics of the Third Joint
def T3(theta_1,theta_2,theta_3,L_1,L_2,L_3,s):
    return T(theta_1,L_1) @ T(theta_2,L_2) @ T(theta_3,s*L_3) 

In [6]:
FK1 = (T1(theta_1,L_1,B) @ M) [:-1,0] 
FK1

Matrix([
[B*l_1*cos(theta_1(t))],
[B*l_1*sin(theta_1(t))]])

In [7]:
FK2 = simplify(T2(theta_1,theta_2,L_1,L_2,B) @ M) [:-1,0] 
FK2

Matrix([
[B*l_2*cos(theta_1(t) + theta_2(t)) + l_1*cos(theta_1(t))],
[B*l_2*sin(theta_1(t) + theta_2(t)) + l_1*sin(theta_1(t))]])

In [8]:
FK3 = simplify(T3(theta_1,theta_2,theta_3,L_1,L_2,L_3,B) @ M) [:-1,0] 
FK3

Matrix([
[B*l_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) + l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t))],
[B*l_3*sin(theta_1(t) + theta_2(t) + theta_3(t)) + l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t))]])

### Jacobian

In [9]:
# I am extracting the position part of each translation matrix for the jacobian
X1 = sp.Matrix([theta_1])
J1 = FK1.jacobian(X1)
J1

Matrix([
[-B*l_1*sin(theta_1(t))],
[ B*l_1*cos(theta_1(t))]])

In [10]:
X2 = sp.Matrix([theta_1,theta_2])
J2 = FK2.jacobian(X2)
J2

Matrix([
[-B*l_2*sin(theta_1(t) + theta_2(t)) - l_1*sin(theta_1(t)), -B*l_2*sin(theta_1(t) + theta_2(t))],
[ B*l_2*cos(theta_1(t) + theta_2(t)) + l_1*cos(theta_1(t)),  B*l_2*cos(theta_1(t) + theta_2(t))]])

In [11]:
X3 = sp.Matrix([theta_1,theta_2,theta_3])
J3 = simplify(FK3.jacobian(X3))
J3

Matrix([
[-B*l_3*sin(theta_1(t) + theta_2(t) + theta_3(t)) - l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)), -B*l_3*sin(theta_1(t) + theta_2(t) + theta_3(t)) - l_2*sin(theta_1(t) + theta_2(t)), -B*l_3*sin(theta_1(t) + theta_2(t) + theta_3(t))],
[ B*l_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) + l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)),  B*l_3*cos(theta_1(t) + theta_2(t) + theta_3(t)) + l_2*cos(theta_1(t) + theta_2(t)),  B*l_3*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

In [26]:
def nT(theta,L):
    return np.array([[np.cos(theta), -np.sin(theta), L*np.cos(theta)],
                     [np.sin(theta),  np.cos(theta), L*np.sin(theta)],
                     [            0,              0,               1]])

def nT1(theta_1,L_1,s):
    return nT(theta_1,L_1*s)

def nT2(theta_1,theta_2,L_1,L_2,s):
    return nT(theta_1,L_1) @ nT(theta_2,L_2*s)

def nT3(theta_1,theta_2,theta_3,L_1,L_2,L_3,s):
    return nT(theta_1,L_1) @ nT(theta_2,L_2) @ nT(theta_3,L_3*s)

def flexible_arm_FK(THETA,LINKS,S):
    theta1,theta2,theta3 = THETA
    link1, link2, link3 = LINKS
    s1,s2,s3 = S
    p0 = np.array([[0],[0],[1]]);
    p1 = nT1(theta1,link1,s1) @ p0
    p2 = nT2(theta1,theta2,link1,link2,s2) @ p0
    p3 = nT3(theta1,theta2,theta3,link1,link2,link3,s3) @ p0;
    return np.hstack([p0[:-1],p1[:-1],p2[:-1],p3[:-1]])

## 2. Joint Motion Function

In [12]:
t = np.linspace(1,10,500)
theta1 = pi/2*np.sin(-t)
theta2 = -pi/2*np.cos(-0.5*t - 4.6)**8 
theta3 = -pi/2*np.cos(-0.5*t - 4.6)**8 

dtheta1 = -pi/2*np.cos(-t)
dtheta2 = -pi/2*np.sin(-0.5*t - 4.6)**8 
dtheta3 = -pi/2*np.sin(-0.5*t - 4.6)**8 

## 3. Evaluating Jacobians at s

In [28]:
S = np.linspace(0.1,1,10)

# Subsituting for Link Lengths
LINKS = [1,1,1]
link1, link2, link3 = LINKS
J1 = J1.subs([[L_1,link1]])
J2 = J2.subs([[L_1,link1],[L_2,link2]])
J3 = J3.subs([[L_1,link1],[L_2,link2],[L_3,link3]])

# Subsituting for s values
J1s = [];
J2s = [];
J3s = [];

for i in range(0,9):
    J1s.append(J1.subs([[B,S[i]]]))
    J2s.append(J2.subs([[B,S[i]]]))
    J3s.append(J3.subs([[B,S[i]]]))
    
# Substituting for theta values
for i in range(0,9):
    J1s[i] = J1s[i].subs([[theta_1,theta1[i]]])
    J2s[i] = J2s[i].subs([[theta_1,theta1[i]],[theta_2,theta2[i]]])
    J3s[i] = J3s[i].subs([[theta_1,theta1[i]],[theta_2,theta2[i]],[theta_3,theta3[i]]])

V1s = [];
V2s = [];
V3s = [];

# Substituting for theta values
for i in range(0,9):
    V1s.append(J1s[i] * dtheta1[i])
    V2s.append(J2s[i] @ np.array([[dtheta1[i]],[dtheta2[i]]]))
    V3s.append(J3s[i] @ np.array([[dtheta1[i]],[dtheta2[i]],[dtheta3[i]]]))
    
# Substituting for theta values
F1s = [];
F2s = [];
F3s = [];

for i in range(0,9):
    F1s.append(-V1s[i])
    F2s.append(-V2s[i])
    F3s.append(-V3s[i])
    

In [29]:
PTS = []
plt.close()
fig = plt.figure(figsize=(12,12))
for i in range(len(t)):
    LINKS = [1,1,1];
    THETA = [theta1[i],theta2[i],theta3[i]]
    plt.clf()
    ax = plt.gca()
    ax.set_xlim([-3.5,3.5])
    ax.set_ylim([-3.5,3.5])
    PTS.append(flexible_arm_FK(THETA,LINKS,[1,1,1]))
    ax.plot(PTS[i][0,:],PTS[i][1,:],'-')
    plt.draw()
    pts = []
    for j in range(9):
        S = [s[j],s[j],s[j]]
        pts.append(flexible_arm_FK(THETA,LINKS,S))

        ax = plt.gca()
        ax.plot(pts[j][0,:],pts[j][1,:],'.g')
        # ax.plot(-pts[0,:],pts[1,:])
        # ax.plot(-pts[0,:],pts[1,:],'.k')

        
        plt.draw()
        plt.show()
    plt.pause(0.001)


NameError: name 's' is not defined

In [19]:
def nT(theta,L):
    return np.array([[np.cos(theta), -np.sin(theta), L*np.cos(theta)],
                     [np.sin(theta),  np.cos(theta), L*np.sin(theta)],
                     [            0,              0,               1]])

def nT1(theta_1,L_1,s):
    return nT(theta_1,L_1*s)

def nT2(theta_1,theta_2,L_1,L_2,s):
    return nT(theta_1,L_1) @ nT(theta_2,L_2*s)

def nT3(theta_1,theta_2,theta_3,L_1,L_2,L_3,s):
    return nT(theta_1,L_1) @ nT(theta_2,L_2) @ nT(theta_3,L_3*s)

def flexible_arm_FK(THETA,LINKS,S):
    theta1,theta2,theta3 = THETA
    link1, link2, link3 = LINKS
    s1,s2,s3 = S
    p0 = np.array([[0],[0],[1]]);
    p1 = nT1(theta1,link1,s1) @ p0
    p2 = nT2(theta1,theta2,link1,link2,s2) @ p0
    p3 = nT3(theta1,theta2,theta3,link1,link2,link3,s3) @ p0;
    return np.hstack([p0[:-1],p1[:-1],p2[:-1],p3[:-1]])

In [47]:
pts[3]

IndexError: list index out of range

range(0, 9)

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])